In [ ]:
!pip install matplotlib-venn
!pip install category_encoders

import numpy as np
import pandas as pd
import re
import pickle
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import pickle

In [ ]:
proj = pd.read_csv("ApartmentRentPrediction_Milestone2.csv")

print(proj["RentCategory"].isnull().sum())

0


In [ ]:
proj.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,RentCategory,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5508832632,housing/rent/apartment,"Two BR - $1,194/mo - Apartment - come and see ...","Come experience. in Cary, NC offering recently...","Clubhouse,Fireplace,Gym,Patio/Deck,Playground,...",2.5,2.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,800,NaN,Cary,NC,35.7585,-78.7783,RentDigs.com,1568755960
1,5664576849,housing/rent/apartment,One BR 640 West Wilson Street,This unit is located at 640 West Wilson Street...,"Cable or Satellite,Dishwasher,Elevator,Garbage...",1.0,1.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,795,640 West Wilson St,Madison,WI,43.0724,-89.4003,RentLingo,1577015508
2,5668619365,housing/rent/apartment,One BR 2777 Sw Archer Rd,"This unit is located at 2777 Sw Archer Rd, Gai...","Basketball,Cable or Satellite,Dishwasher,Inter...",1.0,1.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,560,2777 SW Archer Road,Gainesville,FL,29.6533,-82.3656,RentLingo,1577358921
3,5668632604,housing/rent/apartment,One BR 534-542 Park Avenue,"This unit is located at 534-542 Park Avenue, O...",NaN,1.0,1.0,USD,No,Thumbnail,...,Low Rent,Monthly,600,534-542 Park Ave,Omaha,NE,41.2562,-96.0404,RentLingo,1577359849
4,5668637441,housing/rent/apartment,Three BR 2216 S Palm Grove Ave,"This unit is located at 2216 S Palm Grove Ave,...",NaN,3.0,3.0,USD,No,Thumbnail,...,High Rent,Monthly,1600,2216 S Palm Grove Avenue,Los Angeles,CA,34.0372,-118.2972,RentLingo,1577360159


In [ ]:
#proj = proj.dropna(subset=["RentCategory"])
x = proj.loc[:, proj.columns != "RentCategory"]

In [ ]:
#time malosh lazma
x['time'] = pd.to_datetime(x['time']).dt.time
y = proj["RentCategory"]

<ipython-input-29-d1d18ad4d29b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['time'] = pd.to_datetime(x['time']).dt.time


In [ ]:
x['time']

0       00:00:01.568755
1       00:00:01.577015
2       00:00:01.577358
3       00:00:01.577359
4       00:00:01.577360
             ...       
8995    00:00:01.568781
8996    00:00:01.577358
8997    00:00:01.577358
8998    00:00:01.568772
8999    00:00:01.577359
Name: time, Length: 9000, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state=42)

In [ ]:
proj['pets_allowed']

0       Cats,Dogs
1       Cats,Dogs
2       Cats,Dogs
3       Cats,Dogs
4             NaN
          ...    
8995          NaN
8996         Cats
8997    Cats,Dogs
8998    Cats,Dogs
8999    Cats,Dogs
Name: pets_allowed, Length: 9000, dtype: object

In [ ]:
print(proj.isnull().sum())

id                 0
category           0
title              0
body               0
amenities       3185
bathrooms         30
bedrooms           7
currency           0
fee                0
has_photo          0
pets_allowed    3751
RentCategory       0
price_type         0
square_feet        0
address         2971
cityname          66
state             66
latitude           7
longitude          7
source             0
time               0
dtype: int64


In [ ]:
import datetime
proj['time'] = datetime.datetime.now().time()
proj.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,RentCategory,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5508832632,housing/rent/apartment,"Two BR - $1,194/mo - Apartment - come and see ...","Come experience. in Cary, NC offering recently...","Clubhouse,Fireplace,Gym,Patio/Deck,Playground,...",2.5,2.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,800,NaN,Cary,NC,35.7585,-78.7783,RentDigs.com,11:37:56.515985
1,5664576849,housing/rent/apartment,One BR 640 West Wilson Street,This unit is located at 640 West Wilson Street...,"Cable or Satellite,Dishwasher,Elevator,Garbage...",1.0,1.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,795,640 West Wilson St,Madison,WI,43.0724,-89.4003,RentLingo,11:37:56.515985
2,5668619365,housing/rent/apartment,One BR 2777 Sw Archer Rd,"This unit is located at 2777 Sw Archer Rd, Gai...","Basketball,Cable or Satellite,Dishwasher,Inter...",1.0,1.0,USD,No,Thumbnail,...,Medium-Priced Rent,Monthly,560,2777 SW Archer Road,Gainesville,FL,29.6533,-82.3656,RentLingo,11:37:56.515985
3,5668632604,housing/rent/apartment,One BR 534-542 Park Avenue,"This unit is located at 534-542 Park Avenue, O...",NaN,1.0,1.0,USD,No,Thumbnail,...,Low Rent,Monthly,600,534-542 Park Ave,Omaha,NE,41.2562,-96.0404,RentLingo,11:37:56.515985
4,5668637441,housing/rent/apartment,Three BR 2216 S Palm Grove Ave,"This unit is located at 2216 S Palm Grove Ave,...",NaN,3.0,3.0,USD,No,Thumbnail,...,High Rent,Monthly,1600,2216 S Palm Grove Avenue,Los Angeles,CA,34.0372,-118.2972,RentLingo,11:37:56.515985


In [ ]:
import datetime
def replace_nulls(df, type='test', X_trainApply=X_train):
  if type == 'train':
    numeric_columns = df.select_dtypes(include=['number']).columns
    string_columns = df.select_dtypes(exclude=['number']).columns
    for ncol in numeric_columns:
        df[ncol] = df[ncol].fillna(df[ncol].mean())
    for scol in string_columns:
        if scol == 'time':
          df['time'] = datetime.datetime.now().time()
        else:
          df[scol] = df[scol].fillna(df[scol].mode()[0])
  else :
    numeric_columns = df.select_dtypes(include=['number']).columns
    string_columns = df.select_dtypes(exclude=['number']).columns
    for ncol in numeric_columns:
        df[ncol] = df[ncol].fillna(X_trainApply[ncol].mean())
    for scol in string_columns:
        if scol == 'time':
          df['time'] = datetime.datetime.now().time()
        else:
          df[scol] = df[scol].fillna(X_trainApply[scol].mode()[0])

replace_nulls(X_train, 'train')
#replace_nulls(X_test)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7200 entries, 6317 to 7270
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7200 non-null   int64  
 1   category      7200 non-null   object 
 2   title         7200 non-null   object 
 3   body          7200 non-null   object 
 4   amenities     7200 non-null   object 
 5   bathrooms     7200 non-null   float64
 6   bedrooms      7200 non-null   float64
 7   currency      7200 non-null   object 
 8   fee           7200 non-null   object 
 9   has_photo     7200 non-null   object 
 10  pets_allowed  7200 non-null   object 
 11  price_type    7200 non-null   object 
 12  square_feet   7200 non-null   int64  
 13  address       7200 non-null   object 
 14  cityname      7200 non-null   object 
 15  state         7200 non-null   object 
 16  latitude      7200 non-null   float64
 17  longitude     7200 non-null   float64
 18  source        7200 non-null   

In [ ]:
print(x.nunique())

id              9000
category           3
title           8441
body            8969
amenities       2093
bathrooms         14
bedrooms          10
currency           1
fee                1
has_photo          3
pets_allowed       3
price_type         3
square_feet     1682
address         6016
cityname        1497
state             50
latitude        2257
longitude       2254
source            11
time             142
dtype: int64


In [ ]:
print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9000 non-null   int64  
 1   category      9000 non-null   object 
 2   title         9000 non-null   object 
 3   body          9000 non-null   object 
 4   amenities     5815 non-null   object 
 5   bathrooms     8970 non-null   float64
 6   bedrooms      8993 non-null   float64
 7   currency      9000 non-null   object 
 8   fee           9000 non-null   object 
 9   has_photo     9000 non-null   object 
 10  pets_allowed  5249 non-null   object 
 11  price_type    9000 non-null   object 
 12  square_feet   9000 non-null   int64  
 13  address       6029 non-null   object 
 14  cityname      8934 non-null   object 
 15  state         8934 non-null   object 
 16  latitude      8993 non-null   float64
 17  longitude     8993 non-null   float64
 18  source        9000 non-null 

In [ ]:
def replace_outliers(df,type='train',X_train=X_train):
  numeric_columns = df.select_dtypes(include=['number']).columns
  if type=='train':
     for col in numeric_columns:
      Q1 = df[col].quantile(0.25)
      Q3 = df[col].quantile(0.75)
      IQR = Q3 - Q1
      df.loc[:,col] = np.where((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)), Q3, df.loc[:,col])
  else:
    for col in numeric_columns:
      Q1 = X_train[col].quantile(0.25)
      Q3 = X_train[col].quantile(0.75)
      IQR = Q3 - Q1
      df.loc[:,col] = np.where((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)), Q3, df.loc[:,col])

# replace_outliers(X_train)
# replace_outliers(X_test,'test')
print(x.nunique())

id              9000
category           3
title           8441
body            8969
amenities       2093
bathrooms         14
bedrooms          10
currency           1
fee                1
has_photo          3
pets_allowed       3
price_type         3
square_feet     1682
address         6016
cityname        1497
state             50
latitude        2257
longitude       2254
source            11
time             142
dtype: int64


In [ ]:
# outlier_detection(X_test)
# outlier_detection(X_train)

In [ ]:
x

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5508832632,housing/rent/apartment,"Two BR - $1,194/mo - Apartment - come and see ...","Come experience. in Cary, NC offering recently...","Clubhouse,Fireplace,Gym,Patio/Deck,Playground,...",2.5,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,800,NaN,Cary,NC,35.7585,-78.7783,RentDigs.com,00:00:01.568755
1,5664576849,housing/rent/apartment,One BR 640 West Wilson Street,This unit is located at 640 West Wilson Street...,"Cable or Satellite,Dishwasher,Elevator,Garbage...",1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,795,640 West Wilson St,Madison,WI,43.0724,-89.4003,RentLingo,00:00:01.577015
2,5668619365,housing/rent/apartment,One BR 2777 Sw Archer Rd,"This unit is located at 2777 Sw Archer Rd, Gai...","Basketball,Cable or Satellite,Dishwasher,Inter...",1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,560,2777 SW Archer Road,Gainesville,FL,29.6533,-82.3656,RentLingo,00:00:01.577358
3,5668632604,housing/rent/apartment,One BR 534-542 Park Avenue,"This unit is located at 534-542 Park Avenue, O...",NaN,1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,600,534-542 Park Ave,Omaha,NE,41.2562,-96.0404,RentLingo,00:00:01.577359
4,5668637441,housing/rent/apartment,Three BR 2216 S Palm Grove Ave,"This unit is located at 2216 S Palm Grove Ave,...",NaN,3.0,3.0,USD,No,Thumbnail,NaN,Monthly,1600,2216 S Palm Grove Avenue,Los Angeles,CA,34.0372,-118.2972,RentLingo,00:00:01.577360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,5509251724,housing/rent/apartment,Prominence Apartments Two BR Luxury Apartment ...,Price from: 1158. Your hunt for apartments to ...,"Clubhouse,Gym,Patio/Deck,Pool",1.0,2.0,USD,No,Thumbnail,NaN,Monthly,875,NaN,New London,CT,41.3498,-72.1062,RentDigs.com,00:00:01.568781
8996,5668612787,housing/rent/apartment,One BR 10001 Ne 1st St,"This unit is located at 10001 Ne first St, Bel...",NaN,1.0,1.0,USD,No,Thumbnail,Cats,Monthly,824,10001 NE 1st St,Bellevue,WA,47.5988,-122.1507,RentLingo,00:00:01.577358
8997,5668617853,housing/rent/apartment,Two BR 95 E 2nd St,"This unit is located at 95 E second St, Fond D...","Parking,Storage",1.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,844,95 E 2nd St,Fond Du Lac,WI,43.7815,-88.4411,RentLingo,00:00:01.577358
8998,5509093066,housing/rent/apartment,Pico Rivera - superb Apartment nearby fine din...,Top Rated by Yelp and 5 yrs in a Row! Style an...,"AC,Clubhouse,Dishwasher,Gated,Gym,Internet Acc...",1.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,489,NaN,Pico Rivera,CA,33.9883,-118.0875,RentDigs.com,00:00:01.568772


In [ ]:
# def get_categorical_features_with_low_cardinality(df):
#   categorical_features = {}
#   for col in df.columns:
#     if df[col].dtype == 'object' and df[col].nunique() <= 10:
#       categorical_features[col] = df[col].unique().tolist()
#   return categorical_features

In [ ]:
# get_categorical_features_with_low_cardinality(x)

In [ ]:
# def one_hot_encode_column(df, column_name):
#   # Check if the column exists
#   if column_name not in df.columns:
#     raise ValueError(f"Column '{column_name}' not found in the DataFrame.")
#   # Get dummies for the column
#   dummies = pd.get_dummies(df[column_name], prefix=column_name).replace({True: 1, False: 0})
#   # Concatenate the original DataFrame with the dummies
#   df = pd.concat([df, dummies], axis=1)
#   # Drop the original column (optional)
#   df = df.drop(column_name, axis=1)  # Uncomment to drop the original column
#   return df


In [ ]:
# def update_dataframe_based_on_column_value_loop(df, target_column, col1_name, col2_name):
#   # Check if columns exist
#   if not all(col in df.columns for col in [target_column, col1_name, col2_name]):
#     raise ValueError("One or more column names are not present in the DataFrame.")
#   # Iterate through rows in the target column
#   for index, row in df.iterrows():
#     if row[target_column] == 1:
#       df.at[index, col1_name] = 1
#       df.at[index, col2_name] = 1
#   df.drop(target_column, axis=1, inplace=True)
#   return df

In [ ]:

# def label_encode_column(df, column_name, encoding_type="train", X_train=X_train):
#   # Check if the column exists
#   if column_name not in df.columns:
#       raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

#   # Create a label encoder
#   encoder = LabelEncoder()

#   if encoding_type == "train":
#       # Train mode: encode using fit_transform on the column itself
#       df[column_name] = encoder.fit_transform(df[column_name])

#   elif encoding_type == "test":

#       # Ensure X_train has the column to be encoded
#       if column_name not in X_train.columns:
#           raise ValueError(f"Column '{column_name}' not found in X_train")
#       # Fit the encoder on the training data
#       encoder.fit(X_train[column_name])
#       # Transform the column in the test DataFrame using the fitted encoder
#       df[column_name] = encoder.transform(df[column_name])

#   else:
#       raise ValueError(f"Invalid encoding_type: {encoding_type}")

#   return df

In [ ]:
# x[['pets_allowed','price_type']].head(30)

In [ ]:
# x['address']
# x['body']
# x['amenities']
# x['category']
# x['cityname']
# x['state']
# x['source']

In [ ]:
# X_train['bathrooms'] = np.ceil(X_train['bathrooms'].astype(int))
# X_test['bathrooms'] = np.ceil(X_test['bathrooms'].astype(int))

In [ ]:
# x.columns.tolist()

In [ ]:
# if 'address' in x.columns:
#     print('The variable "address" is present in the DataFrame.')
# else:
#     print('The variable "address" is not present in the DataFrame.')

In [ ]:
X_train

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
6317,5668624413,housing/rent/apartment,Three BR 2314 Pleasure Run Drive,This unit is located at 2314 Pleasure Run Driv...,"Garbage Disposal,Refrigerator",2.0,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1459,2314 Pleasure Run Dr,Ruskin,FL,27.7164,-82.4294,RentLingo,11:37:56.593028
740,5664573224,housing/rent/apartment,Three BR 121 Omega Court,"This unit is located at 121 Omega Court, Dalla...","Dishwasher,Refrigerator",2.0,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1581,121 Omega CT,Dallas,GA,33.9217,-84.8634,RentLingo,11:37:56.593028
3781,5508812979,housing/rent/apartment,"Attractive Two BR, Two BA","Square footage: 1123 sq-ft, unit number: 1038....",Parking,2.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1123,500 Sleepy Hollow Rd,Las Vegas,NV,36.1536,-115.1965,RentDigs.com,11:37:56.593028
7850,5664596563,housing/rent/apartment,Three BR 209 BA St,"This unit is located at 209 Bathrooms St, Sant...","Dishwasher,Refrigerator",2.5,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,2207,209 Bath St,Santa Barbara,CA,34.4312,-119.7292,RentLingo,11:37:56.593028
2963,5668639298,housing/rent/apartment,Studio apartment 1385 NW 22nd Ave.,"This unit is located at 1385 NW 22nd Ave., Por...","Dishwasher,Refrigerator",1.0,0.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,350,1385 NW 22nd Avenue,Portland,OR,45.5091,-122.6449,RentLingo,11:37:56.593028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5668626499,housing/rent/apartment,Three BR 7305 16th Avenue Northwest Unit B,This unit is located at 7305 16th Avenue North...,"Dishwasher,Refrigerator",1.0,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1700,7305 16th Ave Northwest Unit B,Seattle,WA,47.6160,-122.3275,RentLingo,11:37:56.593028
5191,5664570303,housing/rent/apartment,One BR 1001 Leah Avenue,"This unit is located at 1001 Leah Avenue, San ...",Dishwasher,1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,715,1001 Leah Ave,San Marcos,TX,29.8831,-97.9450,RentLingo,11:37:56.593028
5390,5509091562,housing/rent/apartment,"One BA $1,025/mo Arlington - must see to belie...",Feel the energy of the GRIDs vibrant social sc...,"Gym,Parking",1.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,803,500 Sleepy Hollow Rd,Arlington,TX,32.7583,-97.0880,RentDigs.com,11:37:56.593028
860,5664579706,housing/rent/apartment,One BR 402 Genesee Street,"This unit is located at 402 Genesee Street, De...","Dishwasher,Garbage Disposal,Parking,Patio/Deck...",1.0,1.0,USD,No,Thumbnail,Cats,Monthly,758,402 Genesee St,Delafield,WI,43.0619,-88.4017,RentLingo,11:37:56.593028


In [ ]:
# proj.drop(columns=["time"], inplace=True)
# #proj.head()
# x.drop(columns=["time"], inplace=True)
X_train.drop(columns=["time"], inplace=True)
# X_test = X_test.loc[:, X_test.columns != "time"]

In [ ]:
X_train.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source
6317,5668624413,housing/rent/apartment,Three BR 2314 Pleasure Run Drive,This unit is located at 2314 Pleasure Run Driv...,"Garbage Disposal,Refrigerator",2.0,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1459,2314 Pleasure Run Dr,Ruskin,FL,27.7164,-82.4294,RentLingo
740,5664573224,housing/rent/apartment,Three BR 121 Omega Court,"This unit is located at 121 Omega Court, Dalla...","Dishwasher,Refrigerator",2.0,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1581,121 Omega CT,Dallas,GA,33.9217,-84.8634,RentLingo
3781,5508812979,housing/rent/apartment,"Attractive Two BR, Two BA","Square footage: 1123 sq-ft, unit number: 1038....",Parking,2.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1123,500 Sleepy Hollow Rd,Las Vegas,NV,36.1536,-115.1965,RentDigs.com
7850,5664596563,housing/rent/apartment,Three BR 209 BA St,"This unit is located at 209 Bathrooms St, Sant...","Dishwasher,Refrigerator",2.5,3.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,2207,209 Bath St,Santa Barbara,CA,34.4312,-119.7292,RentLingo
2963,5668639298,housing/rent/apartment,Studio apartment 1385 NW 22nd Ave.,"This unit is located at 1385 NW 22nd Ave., Por...","Dishwasher,Refrigerator",1.0,0.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,350,1385 NW 22nd Avenue,Portland,OR,45.5091,-122.6449,RentLingo


In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

cols=['pets_allowed', 'price_type','fee','currency','has_photo','amenities', 'address', 'body', 'category', 'cityname', 'state', 'source', 'title']
label_encoders = {}

for col in cols:
    label_encoder = LabelEncoder()
    X_train[col] = label_encoder.fit_transform(X_train[col])
    label_encoders[col] = label_encoder

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# with open('label_encoders.pkl', 'rb') as f:
#     label_encoders = pickle.load(f)

# for col in cols:
#     X_test[col] = X_test[col].apply(lambda s: label_encoders[col].transform([s])[0] if s in label_encoders[col].classes_ else s)

# for col in cols:
#   max_value_train = X_train[col].max()
#   unique_values_test = X_test[col].unique()
#   for value in unique_values_test:
#     if value not in X_train[col].unique():
#       max_value_train += 1
#       X_test.loc[X_test[col] == value, col] = max_value_train

y_label_encoder = LabelEncoder()
y_train =pd.DataFrame( y_label_encoder.fit_transform(y_train))
with open('y_label_encoder.pkl', 'wb') as f:
    pickle.dump(y_label_encoder, f)
# le = pickle.load(open('y_label_encoder.pkl', 'rb'))

# y_test =pd.DataFrame(le.transform(y_test))

In [ ]:
y_train

,0
0,2
1,2
2,2
3,0
4,2
...,...
7195,0
7196,1
7197,2
7198,2


In [ ]:
y_test

7940    Medium-Priced Rent
1162    Medium-Priced Rent
582     Medium-Priced Rent
4081              Low Rent
8412    Medium-Priced Rent
               ...        
3063              Low Rent
6925    Medium-Priced Rent
6521              Low Rent
3002             High Rent
1298             High Rent
Name: RentCategory, Length: 1800, dtype: object

In [ ]:
# for col in cols:
#     max_value_train = X_train[col].max()
#     unique_values_test = X_test[col].unique()
#     for value in unique_values_test:
#         if value not in X_train[col].unique():
#             max_value_train += 1
#             X_test.loc[X_test[col] == value, col] = max_value_train

In [ ]:
X_train.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source
6317,5668624413,0,5038,3679,1570,2.0,3.0,0,0,1,1,0,1459,1892,1056,9,27.7164,-82.4294,8
740,5664573224,0,4846,2415,1431,2.0,3.0,0,0,1,1,0,1581,519,303,10,33.9217,-84.8634,8
3781,5508812979,0,408,1168,1716,2.0,2.0,0,0,1,1,0,1123,3387,633,32,36.1536,-115.1965,7
7850,5664596563,0,5012,3449,1431,2.5,3.0,0,0,1,1,0,2207,1645,1094,4,34.4312,-119.7292,8
2963,5668639298,0,4164,2690,1431,1.0,0.0,0,0,1,1,0,350,810,979,36,45.5091,-122.6449,8


In [ ]:
X_test.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
7940,5509201929,housing/rent/apartment,Pet Friendly 2+2 Apartment in Odenton. Single ...,"Square footage: 1058 square feet, unit number:...","AC,Cable or Satellite,Clubhouse,Dishwasher,Ele...",2.0,2.0,USD,No,Yes,"Cats,Dogs",Monthly,1058,NaN,Odenton,MD,39.0747,-76.7017,RentDigs.com,00:00:01.568779
1162,5668627277,housing/rent/apartment,Two BR 6343 N Wayne Ave,"This unit is located at 6343 N Wayne Ave, Chic...",NaN,1.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1400,6343 N Wayne Avenue,Chicago,IL,41.8625,-87.6825,RentLingo,00:00:01.577359
582,5508866806,housing/rent/apartment,"Bright Denver, One BR, One BA for rent. Offstr...","Square footage: 630 sq. feet, unit number: 05....","Fireplace,Parking,Patio/Deck,Storage,Washer Dryer",1.0,1.0,USD,No,Thumbnail,NaN,Monthly,630,NaN,Denver,CO,39.6767,-104.8854,RentDigs.com,00:00:01.568758
4081,5651549929,housing/rent/apartment,10891 Edgemere Blvd J8,SPACIOUS 2 BEDROOMs APARTMENT WITH WASHER+DRYE...,"Cable or Satellite,Dishwasher,Garbage Disposal...",1.0,2.0,USD,No,Thumbnail,NaN,Monthly,840,10891 Edgemere Boulevard J8,El Paso,TX,31.7951,-106.3886,ListedBuy,00:00:01.576189
8412,5668634408,housing/rent/apartment,Two BR 15919 172nd Street SE,"This unit is located at 15919 172nd Street SE,...",NaN,1.0,2.0,USD,No,Thumbnail,Cats,Monthly,600,15919 172nd St SE,Snohomish,WA,47.8920,-122.0581,RentLingo,00:00:01.577359


In [ ]:
# def feature_scaling(df, scaler_type='standard',type='test',X_train=X_train):
#   # Check scaler type
#   if scaler_type not in ['standard', 'minmax']:
#     raise ValueError("Invalid scaler type. Choose 'standard' or 'minmax'.")
#   # Create a scaler object
#   #Use StandardScaler when your data features have different units or different scales, and when you assume a Gaussian distribution.
#   if scaler_type == 'standard':
#     scaler = StandardScaler()
#   else:
#     # If your features have known bounds, such as percentages (ranging from 0 to 100%)
#     scaler = MinMaxScaler()
#   if type == 'train':
#      scaler.fit(df)
#      df_scaled = pd.DataFrame(scaler.transform(df), columns=df.columns)
#   else:
#     scaler.fit(X_train)
#     df_scaled = pd.DataFrame(scaler.transform(df), columns=df.columns)
#   return df_scaled

In [ ]:
def feature_scaling(df,type='test',X_train=X_train):
  # Check scaler type  , scaler_type='standard'
    scaler = StandardScaler()
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df), columns=df.columns)
    with open('feature_scaler.pkl', 'wb') as f:
      pickle.dump(scaler, f)

In [ ]:
feature_scaling(X_train,'train')
# X_test = feature_scaling(X_test,'standard','test')

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7200 entries, 6317 to 7270
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7200 non-null   int64  
 1   category      7200 non-null   int64  
 2   title         7200 non-null   int64  
 3   body          7200 non-null   int64  
 4   amenities     7200 non-null   int64  
 5   bathrooms     7200 non-null   float64
 6   bedrooms      7200 non-null   float64
 7   currency      7200 non-null   int64  
 8   fee           7200 non-null   int64  
 9   has_photo     7200 non-null   int64  
 10  pets_allowed  7200 non-null   int64  
 11  price_type    7200 non-null   int64  
 12  square_feet   7200 non-null   int64  
 13  address       7200 non-null   int64  
 14  cityname      7200 non-null   int64  
 15  state         7200 non-null   int64  
 16  latitude      7200 non-null   float64
 17  longitude     7200 non-null   float64
 18  source        7200 non-null   

In [ ]:
X_test.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
7940,5509201929,housing/rent/apartment,Pet Friendly 2+2 Apartment in Odenton. Single ...,"Square footage: 1058 square feet, unit number:...","AC,Cable or Satellite,Clubhouse,Dishwasher,Ele...",2.0,2.0,USD,No,Yes,"Cats,Dogs",Monthly,1058,NaN,Odenton,MD,39.0747,-76.7017,RentDigs.com,00:00:01.568779
1162,5668627277,housing/rent/apartment,Two BR 6343 N Wayne Ave,"This unit is located at 6343 N Wayne Ave, Chic...",NaN,1.0,2.0,USD,No,Thumbnail,"Cats,Dogs",Monthly,1400,6343 N Wayne Avenue,Chicago,IL,41.8625,-87.6825,RentLingo,00:00:01.577359
582,5508866806,housing/rent/apartment,"Bright Denver, One BR, One BA for rent. Offstr...","Square footage: 630 sq. feet, unit number: 05....","Fireplace,Parking,Patio/Deck,Storage,Washer Dryer",1.0,1.0,USD,No,Thumbnail,NaN,Monthly,630,NaN,Denver,CO,39.6767,-104.8854,RentDigs.com,00:00:01.568758
4081,5651549929,housing/rent/apartment,10891 Edgemere Blvd J8,SPACIOUS 2 BEDROOMs APARTMENT WITH WASHER+DRYE...,"Cable or Satellite,Dishwasher,Garbage Disposal...",1.0,2.0,USD,No,Thumbnail,NaN,Monthly,840,10891 Edgemere Boulevard J8,El Paso,TX,31.7951,-106.3886,ListedBuy,00:00:01.576189
8412,5668634408,housing/rent/apartment,Two BR 15919 172nd Street SE,"This unit is located at 15919 172nd Street SE,...",NaN,1.0,2.0,USD,No,Thumbnail,Cats,Monthly,600,15919 172nd St SE,Snohomish,WA,47.8920,-122.0581,RentLingo,00:00:01.577359


In [ ]:
X_train.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price_type,square_feet,address,cityname,state,latitude,longitude,source
6317,5668624413,0,5038,3679,1570,2.0,3.0,0,0,1,1,0,1459,1892,1056,9,27.7164,-82.4294,8
740,5664573224,0,4846,2415,1431,2.0,3.0,0,0,1,1,0,1581,519,303,10,33.9217,-84.8634,8
3781,5508812979,0,408,1168,1716,2.0,2.0,0,0,1,1,0,1123,3387,633,32,36.1536,-115.1965,7
7850,5664596563,0,5012,3449,1431,2.5,3.0,0,0,1,1,0,2207,1645,1094,4,34.4312,-119.7292,8
2963,5668639298,0,4164,2690,1431,1.0,0.0,0,0,1,1,0,350,810,979,36,45.5091,-122.6449,8


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Initialize the ANOVA selector
anova_selector = SelectKBest(score_func=f_classif, k=15)

# Fit the selector on the training data
anova_selector.fit(X_train, y_train)
x_train = X_train
X_train = pd.DataFrame(anova_selector.transform(X_train))

with open('feature_selection.pkl', 'wb') as f:
     pickle.dump(anova_selector, f)

print(X_train)

                0    1       2       3       4    5    6    7    8       9   \
0     5.668624e+09  0.0  5038.0  3679.0  1570.0  2.0  3.0  1.0  1.0  1459.0   
1     5.664573e+09  0.0  4846.0  2415.0  1431.0  2.0  3.0  1.0  1.0  1581.0   
2     5.508813e+09  0.0   408.0  1168.0  1716.0  2.0  2.0  1.0  1.0  1123.0   
3     5.664597e+09  0.0  5012.0  3449.0  1431.0  2.5  3.0  1.0  1.0  2207.0   
4     5.668639e+09  0.0  4164.0  2690.0  1431.0  1.0  0.0  1.0  1.0   350.0   
...            ...  ...     ...     ...     ...  ...  ...  ...  ...     ...   
7195  5.668626e+09  0.0  5347.0  5863.0  1431.0  1.0  3.0  1.0  1.0  1700.0   
7196  5.664570e+09  0.0  1418.0  1984.0  1214.0  1.0  1.0  1.0  1.0   715.0   
7197  5.509092e+09  0.0  1340.0   520.0  1616.0  1.0  2.0  1.0  1.0   803.0   
7198  5.664580e+09  0.0  2712.0  4691.0  1318.0  1.0  1.0  1.0  0.0   758.0   
7199  5.668625e+09  0.0  2684.0  4640.0   895.0  1.0  1.0  1.0  1.0   825.0   

          10      11    12       13        14  
0  

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [7 8] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9000 non-null   int64  
 1   category      9000 non-null   object 
 2   title         9000 non-null   object 
 3   body          9000 non-null   object 
 4   amenities     5815 non-null   object 
 5   bathrooms     8970 non-null   float64
 6   bedrooms      8993 non-null   float64
 7   currency      9000 non-null   object 
 8   fee           9000 non-null   object 
 9   has_photo     9000 non-null   object 
 10  pets_allowed  5249 non-null   object 
 11  price_type    9000 non-null   object 
 12  square_feet   9000 non-null   int64  
 13  address       6029 non-null   object 
 14  cityname      8934 non-null   object 
 15  state         8934 non-null   object 
 16  latitude      8993 non-null   float64
 17  longitude     8993 non-null   float64
 18  source        9000 non-null 

In [ ]:
print(y_train)

      0
0     2
1     2
2     2
3     0
4     2
...  ..
7195  0
7196  1
7197  2
7198  2
7199  0

[7200 rows x 1 columns]


In [ ]:
y_train

,0
0,2
1,2
2,2
3,0
4,2
...,...
7195,0
7196,1
7197,2
7198,2


In [ ]:
y_test

7940    Medium-Priced Rent
1162    Medium-Priced Rent
582     Medium-Priced Rent
4081              Low Rent
8412    Medium-Priced Rent
               ...        
3063              Low Rent
6925    Medium-Priced Rent
6521              Low Rent
3002             High Rent
1298             High Rent
Name: RentCategory, Length: 1800, dtype: object

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LogisticRegression_model = LogisticRegression().fit(X_train,y_train)
y_pred = LogisticRegression_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("TRAINING Logistic Reg accuracy:", accuracy)

pickle.dump(LogisticRegression_model,open('/content/LogisticRegression_model_saved','wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TRAINING Logistic Reg accuracy: 0.54125


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
# Create an SVM classifier
rbf_svc_model = svm.SVC(kernel='rbf', gamma = 1 , C=3) #BEST one //edited// gamma =0.8 & C = 1
rbf_svc_model.fit(X_train, y_train)
y_train_pred = rbf_svc_model.predict(X_train)

train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)
pickle.dump(rbf_svc_model,open('/content/rbf_svc_model_saved','wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Accuracy: 0.99875


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

RandomForest = RandomForestClassifier(n_estimators=100, random_state=42)
RandomForest.fit(X_train, y_train)
y_pred_train = RandomForest.predict(X_train)
# y_pred_test = RandomForest.predict(X_test)
accuracy_train = accuracy_score(y_train, y_pred_train)
# accuracy_test = accuracy_score(y_test, y_pred_test)
print("Training Accuracy:", accuracy_train)
# print("Test Accuracy:", accuracy_test)


pickle.dump(RandomForest,open('/content/RandomForest_model_saved','wb'))

<ipython-input-74-fe884938d2ed>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RandomForest.fit(X_train, y_train)


Training Accuracy: 1.0


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

DecisionTreeClassifiermodel = DecisionTreeClassifier()
DecisionTreeClassifiermodel.fit(X_train, y_train)
y_pred_train = DecisionTreeClassifiermodel.predict(X_train)
# y_pred_test = DecisionTreeClassifiermodel.predict(X_test)
accuracy_train = accuracy_score(y_train, y_pred_train)
# accuracy_test = accuracy_score(y_test, y_pred_test)
print("Training Accuracy:", accuracy_train)
# print("Test Accuracy:", accuracy_test)


pickle.dump(DecisionTreeClassifiermodel,open('/content/DecisionTreeClassifiermodel_saved','wb'))

Training Accuracy: 1.0


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_train_pred = knn_model.predict(X_train)
# train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)
# y_test_pred = knn_model.predict(X_test)
# test_accuracy = accuracy_score(y_test, y_test_pred)
# print("Test Accuracy:", test_accuracy)

pickle.dump(knn_model,open('/content/KNN_model_saved','wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Training Accuracy: 0.99875


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score
import numpy as np
# Voting Classifier
voting_clf = VotingClassifier(estimators=[('logistic', LogisticRegression_model), ('svm', rbf_svc_model),('Random Forest', RandomForest),('dicision tree', DecisionTreeClassifiermodel),('KNN_model',knn_model)], voting='hard')
voting_clf.fit(X_train, y_train)
# voting_pred = voting_clf.predict(X_test)
# voting_accuracy = accuracy_score(y_test, voting_pred)
# print("Voting Classifier Accuracy:", voting_accuracy)

pickle.dump(voting_clf,open('/content/Voting_Classifier_saved','wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
# Stacking Classifier
estimators = [('logistic', LogisticRegression_model), ('svm', rbf_svc_model),('Random Forest', RandomForest),('dicision tree', DecisionTreeClassifiermodel),('KNN_model',knn_model)]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

pickle.dump(stacking_clf,open('/content/Stacking_Classifier_saved','wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear

In [ ]:
new_data = pd.read_csv('/content/ApartmentRentPrediction_Milestone2.csv')

with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

cols=['pets_allowed', 'price_type','fee','currency','has_photo','amenities', 'address', 'body', 'category', 'cityname', 'state', 'source', 'title']
for col in cols:
    new_data[col] = new_data[col].apply(lambda s: label_encoders[col].transform([s])[0] if s in label_encoders[col].classes_ else s)

for col in cols:
  max_value_train = x_train[col].max()
  unique_values_test = new_data[col].unique()
  for value in unique_values_test:
    if value not in x_train[col].unique():
      max_value_train += 1
      new_data.loc[new_data[col] == value, col] = max_value_train

In [ ]:
new_data = new_data.dropna(subset=["RentCategory"])
new_data_features = pd.DataFrame(new_data.loc[:, ~new_data.columns.isin(["RentCategory", "time"])])

new_data_target = new_data["RentCategory"]
replace_nulls(new_data_features)

le_encoder_target = pickle.load(open('y_label_encoder.pkl', 'rb'))
new_data_target = le_encoder_target.transform(new_data_target)

le = pickle.load(open('y_label_encoder.pkl', 'rb'))
new_data["RentCategory"] =pd.DataFrame(le.transform(y_test))

feature_scaler = pickle.load(open('feature_scaler.pkl', 'rb'))
new_data_features = feature_scaler.transform(new_data_features)

feature_selection = pickle.load(open('feature_selection.pkl', 'rb'))
new_data_features = feature_selection.transform(new_data_features)
new_data_features = pd.DataFrame(new_data_features)

LR_model = pickle.load(open('/content/LogisticRegression_model_saved','rb'))
LR_pred = LR_model.predict(new_data_features)
print("logistic regression Test Accuracy:", accuracy_score(new_data_target, LR_pred))

rbf_svm = pickle.load(open('/content/rbf_svc_model_saved','rb'))
rbf_pred = rbf_svm.predict(new_data_features)
print("RBF SVM Test Accuracy:", accuracy_score(new_data_target, rbf_pred))

RandForest = pickle.load(open('/content/RandomForest_model_saved','rb'))
rand_pred = RandForest.predict(new_data_features)
print("Random Forest Test Accuracy:", accuracy_score(new_data_target, rand_pred))

decisionTree = pickle.load(open('/content/DecisionTreeClassifiermodel_saved','rb'))
DT_pred = decisionTree.predict(new_data_features)
print("Decision Tree Test Accuracy:", accuracy_score(new_data_target, DT_pred))

KNN_model = pickle.load(open('/content/KNN_model_saved','rb'))
KNN_pred = KNN_model.predict(new_data_features)
print("KNN Classifier Test Accuracy:", accuracy_score(new_data_target, KNN_pred))

voting_classifier = pickle.load(open('/content/Voting_Classifier_saved','rb'))
Voting_pred = voting_classifier.predict(new_data_features)
print("Voting Classifier Test Accuracy:", accuracy_score(new_data_target, Voting_pred))

stacking_classifier = pickle.load(open('/content/Stacking_Classifier_saved','rb'))
stacking_pred = stacking_classifier.predict(new_data_features)
print("Stacking Classifier Test Accuracy:", accuracy_score(new_data_target, stacking_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(


logistic regression Test Accuracy: 0.418
RBF SVM Test Accuracy: 0.5425555555555556
Random Forest Test Accuracy: 0.376
Decision Tree Test Accuracy: 0.5425555555555556
KNN Classifier Test Accuracy: 0.15944444444444444
Voting Classifier Test Accuracy: 0.42033333333333334
Stacking Classifier Test Accuracy: 0.5004444444444445
